# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd

In [2]:
from IPython.display import IFrame
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
IFrame(url, width=800, height=350)

We can use the function read_html instead of BeautifulSoup

In [10]:
data, = pd.read_html(url, match="Postal Code", skiprows=0)
data.columns = ["PostalCode", "Borough", "Neighborhood"]
data.head()
#inserting a coma after data, the problem is solved

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
#We drop columns where Borough is not assigned

data = data[data["Borough"] != "Not assigned"]
data.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma

In [12]:
borough_fun = lambda s: s.iloc[0]
neighborhood_fun = lambda s: ", ".join(s)
agg_funcs = {"Borough": borough_fun, "Neighborhood": neighborhood_fun}
data_temp = data.groupby(by="PostalCode").aggregate(agg_funcs)
data_temp.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [13]:
#Reset INDEX
data = data_temp.reset_index()[data.columns]
data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough. Lets check which cell as Not assigned value

In [15]:
data[data["Neighborhood"] == "Not assigned"]

,PostalCode,Borough,Neighborhood


We dont have so lets continue...

In [16]:
data.shape

(103, 3)

# Latitude and the longitude coordinates of each neighborhood

In [33]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install geopy

     |████████████████████████████████| 112 kB 4.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [40]:
pip install folium

     |████████████████████████████████| 94 kB 2.5 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


We import the csv whit de Latitude and Longitude

In [23]:
locations = pd.read_csv("https://cocl.us/Geospatial_data")
locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We change the column PostalCode to be the same in both tables and then we join both tables

In [25]:
locations.columns = ["PostalCode", "Latitude", "Longitude"]
data2 = pd.merge(data, locations, on='PostalCode')
data2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# EDA

Let's filter only rows where Borough contains word Toronto and explore and cluster that.

In [28]:
subset = data2[data2['Borough'].str.contains("Toronto")]
subset.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [29]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(subset['Borough'].unique()),
        subset.shape[0]
    )
)

The dataframe has 5 boroughs and 40 neighborhoods.


Lets get the geographical data of Toronto

In [41]:
from geopy.geocoders import Nominatim
import folium

In [36]:
address = 'Toronto, CN'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6425637, -79.38708718320467.


In [46]:
# create map of Totonto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(subset['Latitude'], subset['Longitude'], subset['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto